In [2]:
# STANDARD
from pprint import pprint

# PyPI
from bs4 import BeautifulSoup as BS
import requests
from tqdm import tqdm

# LOCAL
from interface_db import db_interface_sqlite as data
from interface_website import website_tools as tools
from interface_website import website_patheos as patheos

In [3]:
with data.database() as db:
    db.create_tables()
    db.insert_update_site_pages(1625, 149)
    result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 47")
    #print(result[0][0])
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

In [4]:
patheos.insert_website('Patheos Blogs', 'https://patheos.com/blogs')
results = patheos.fetch_and_insert_categories('Patheos Blogs')
with data.database() as db:
    result = db.execute("SELECT name FROM categories")
    for i in result:
        results = patheos.fetch_and_insert_blogs(i[0])
        print(f'Inserted: {results.inserted} | Not Inserted: {results.not_inserted} | Errors: {results.exceptions}')

Inserting Patheos Blogs - https://patheos.com/blogs
Insert successful. Name: Patheos Blogs, URL: https://patheos.com/blogs
Pulling categories for Patheos Blogs
buddhist-blogs https://www.patheos.com/buddhist-blogs
Inserted: 5 | Not Inserted: 0 | Errors: 0
catholic-blogs https://www.patheos.com/catholic-blogs
Inserted: 89 | Not Inserted: 0 | Errors: 0
contemplative-blogs https://www.patheos.com/contemplative-blogs
Inserted: 8 | Not Inserted: 0 | Errors: 0
evangelical-blogs https://www.patheos.com/evangelical-blogs
Inserted: 84 | Not Inserted: 0 | Errors: 0
general-christian-blogs https://www.patheos.com/general-christian-blogs
Inserted: 18 | Not Inserted: 0 | Errors: 0
hindu-blogs https://www.patheos.com/hindu-blogs
Inserted: 4 | Not Inserted: 0 | Errors: 0
jewish-blogs https://www.patheos.com/jewish-blogs
Inserted: 6 | Not Inserted: 0 | Errors: 0
latter-day-saint-blogs https://www.patheos.com/latter-day-saint-blogs
Inserted: 17 | Not Inserted: 0 | Errors: 0
muslim-blogs https://www.pat

In [5]:
with data.database() as db:
    result = db.execute("SELECT * FROM blogs WHERE url = 'https://www.patheos.com/blogs/jesuscreed'")
    print(result)
    result = db.execute(f"SELECT number FROM site_pages WHERE site_id = 149")
    print(result)

[(149, 'by Scot McKnight', 'Jesus Creed', 'https://www.patheos.com/blogs/jesuscreed', 4, '2020-04-21', 'default', '2020-04-21', 'default')]
[(1625,)]


In [6]:
import sys

number_of_new_pages = patheos.number_of_blog_pages('Jesus Creed')
print(f'\nNew Pages: {number_of_new_pages}')

1640
New Pages: 15


In [ ]:
import sys

results = tools.insert_results()

with data.database() as db:
    blog = db.query_blogs('Jesus Creed')
    print(blog.id)
resume_page = patheos.find_page_resume_scrape(blog.id)

page_number = resume_page
total = 0
continue_on = True
while continue_on == True:
    base_url = 'https://www.patheos.com/blogs/jesuscreed' + '/page/' + str(page_number)
    results_l = patheos.scrape_posts_on_page(base_url)
    #if results_l != 404:
    for i in results_l:
        post = data.post(i, 149)
        post = patheos.scrape_post(post, 'no')
        if post.title == None:
            print('no title')
        #posts_dict[post.url] = post
        if tools.check_url_new('posts', post.url) == True:
            with data.database() as db:
                # print('\n', post.url)
                db.insert_post(post)
                db.commit()
                results.inserted += 1
        else:
            # <placeholder for logging>
            results.not_inserted += 1
        total += 1
        sys.stdout.write('\r' 'BLOG: ' + blog.name + ' | PAGE: ' + str(page_number) + ' | TOTAL PROCESSED: ' + str(total) 
                                                            + ' | INSERTED: ' + str(results.inserted)
                                                            + ' | NOT INSERTED: ' + str(results.not_inserted))
    #else:
    #    continue_on = False
    page_number += 1
    sys.stdout.write('\r' 'BLOG: ' + blog.name + ' | PAGE: ' + str(page_number) + ' | TOTAL PROCESSED: ' + str(total) 
                                                        + ' | INSERTED: ' + str(results.inserted)
                                                        + ' | NOT INSERTED: ' + str(results.not_inserted))

149
posts 1597
divided 159
BLOG: Jesus Creed | PAGE: 217 | TOTAL PROCESSED: 583 | INSERTED: 566 | NOT INSERTED: 17